In [1]:
import sys
from librosa.core import resample
import pandas as pd
import numpy as np
from IPython.display import Audio
import torch
import pathlib
def create_dir(filename):
    pathlib.Path('/'.join(filename.split('/')[:-1])).mkdir(parents=True, exist_ok=True)
from tqdm.notebook import tqdm
sys.path.append('Conv-TasNet/src/')
sys.path.append('SincNet/')
from conv_tasnet import *
from pit_criterion import cal_loss
from dnn_models import *
from data_io import ReadList,read_conf_inp,str_to_bool
from collections import Counter
import os
device = 0
device_ids = [0, 1, 2, 3]
root = '../'
sr = 8000
torch.cuda.set_device(device)

/home/paperspace/.local/lib/python3.7/site-packages/numba/core/errors.py:144: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)
/home/paperspace/.local/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/paperspace/.local/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
tas2 = ConvTasNet.load_model('final.pth.tar').cuda(device)
tasnet = ConvTasNet(N = 256, L = 20, B = 256, H = 512, P = 3, X = 8, R = 4, C = 3, norm_type="gLN", causal=0,
             mask_nonlinear='relu').cuda(device)

own_state = tasnet.state_dict()
for name, param in tqdm(tas2.state_dict().items()):
    if name not in own_state:
         continue
    if isinstance(param, torch.nn.Parameter):
        param = param.data
    try:
        own_state[name].copy_(param)
    except:
        print('shape mismatch')

optimizer = torch.optim.Adam(tasnet.parameters(), lr = 0.001)

if os.path.exists('models/tasnet.pth'):
    print('load model')
    checkpoint = torch.load('models/tasnet.pth')
    tasnet.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    loss = checkpoint['loss']
    
#tasnet = nn.DataParallel(tasnet, device_ids = device_ids)
tasnet.train()
pass

shape mismatch

load model


In [3]:
class OverlayDataSet(torch.utils.data.Dataset):
    def __init__(self, csv):
        super().__init__()
        self.segments = pd.read_csv(root+csv)
        self.speakers = list(set(self.segments['speaker']))
        self.speakers.sort()
        self.spkr2idx = {spkr:i for i, spkr in enumerate(self.speakers)}

    def __len__(self):
        return len(self.segments)
    def __getitem__(self, idx):
        seg1 = self.segments.iloc[idx]
        seg2 = self.segments.iloc[np.random.randint(len(self.segments))]
        while(seg1['speaker']==seg2['speaker']):
            seg2 = self.segments.iloc[np.random.randint(len(self.segments))]
        seg3 = self.segments.iloc[np.random.randint(len(self.segments))]
        while(seg3['speaker']==seg2['speaker'] or seg3['speaker']==seg1['speaker']):
            seg3 = self.segments.iloc[np.random.randint(len(self.segments))]
        
        
        sig1 = np.load(root+seg1['segfile'])
        sig2 = np.load(root+seg2['segfile'])
        sig3 = np.load(root+seg3['segfile'])

        
        out_vec1 = np.zeros(len(self.speakers)) # maybe try PIT training too
        out_vec2 = np.zeros(len(self.speakers)) # maybe try PIT training too
        out_vec3 = np.zeros(len(self.speakers)) # maybe try PIT training too

        out_vec1[self.spkr2idx[seg1['speaker']]] = 1
        out_vec2[self.spkr2idx[seg2['speaker']]] = 1
        out_vec3[self.spkr2idx[seg3['speaker']]] = 1

        return sig1, sig2, sig3, out_vec1, out_vec2, out_vec3


#mean, std = compute_mean_std('overlay-train.csv')


trainset = OverlayDataSet('train-segments.csv')
valset = OverlayDataSet('val-segments.csv')
testset = OverlayDataSet('test-segments.csv')
idx = np.random.randint(len(testset))

sig1, sig2, sig3, target1, target2, target3 = testset[idx]

mixture = torch.Tensor(sig1+sig2+sig3).cuda(device)
mixture = mixture[None, ...]
out = tasnet(mixture)
new_sig1, new_sig2, new_sig3 = out[0].cpu().detach().numpy()
Audio(new_sig1, rate = sr)

Conv-TasNet/src/utils.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  frame = signal.new_tensor(frame).long()  # signal may in GPU or CPU


In [4]:
Audio(new_sig2, rate = sr)

In [5]:
Audio(new_sig3, rate = sr)

In [6]:
batch_size = 8
trainloader  = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory = True, num_workers = 16)
valloader  = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True, pin_memory = True, num_workers = 16)
tasnet.train()

for epoch in range(64):
    running_loss = 0.0
    for batch_idx, (sig1, sig2, sig3, _, _, _) in enumerate(tqdm(trainloader)):
        optimizer.zero_grad()
        sig1, sig2, sig3 = sig1.float().cuda(device), sig2.float().cuda(device), sig3.float().cuda(device)
        out = tasnet(sig1+sig2+sig3)
        source = torch.stack([sig1, sig2, sig3], dim = 1).detach()
        loss, max_snr, estimate_source, reorder_estimate_source = \
            cal_loss(source, out, torch.ones(sig1.shape[0], dtype = torch.int32).cuda(device)*2*sr) # 2 seconds has 2*sr samples
        loss.backward()
        torch.nn.utils.clip_grad_norm_(tasnet.parameters(), 0.5)
        optimizer.step()

        running_loss += loss.item()
        if batch_idx % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f ' % 
                  (epoch + 1, batch_idx + 1, running_loss / 200))
            running_loss = 0.0
            torch.save({
            'model_state_dict': tasnet.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
            }, 'models/tasnet.pth')
    with torch.no_grad():
        running_loss = 0.0
        tasnet.eval()
        valloader  = torch.utils.data.DataLoader(valset, batch_size=32, shuffle=True, pin_memory = True, num_workers = 16)
        for batch_idx, (sig1, sig2, sig3, _, _, _) in enumerate(tqdm(valloader)):
            sig1, sig2, sig3 = sig1.float().cuda(device), sig2.float().cuda(device), sig3.float().cuda(device)
            out = tasnet(sig1+sig2+sig3)
            source = torch.stack([sig1, sig2, sig3], dim = 1).detach()
            loss, max_snr, estimate_source, reorder_estimate_source = \
                cal_loss(source, out, torch.ones(sig1.shape[0], dtype = torch.int32).cuda(device)*2*sr)
            torch.nn.utils.clip_grad_norm_(tasnet.parameters(), 0.5)

            running_loss += loss.item()*sig1.shape[0]
        print(running_loss/len(valset))

[1,   200] loss: -5.779 
[1,   400] loss: -5.121 
[1,   600] loss: -5.860 
[1,   800] loss: -5.742 
[1,  1000] loss: -5.843 
[1,  1200] loss: -6.186 
[1,  1400] loss: -5.735 
[1,  1600] loss: -5.827 
[1,  1800] loss: -5.976 
[1,  2000] loss: -6.095 
[1,  2200] loss: -5.821 
[1,  2400] loss: -5.802 
[1,  2600] loss: -5.780 
[1,  2800] loss: -5.829 
[1,  3000] loss: -5.726 
[1,  3200] loss: -6.100 
[1,  3400] loss: -5.932 
[1,  3600] loss: -5.762 
[1,  3800] loss: -5.667 
[1,  4000] loss: -5.790 
[1,  4200] loss: -5.918 
[1,  4400] loss: -5.992 
[1,  4600] loss: -5.805 
[1,  4800] loss: -5.858 




-5.760869477705088


[2,   200] loss: -5.995 
[2,   400] loss: -5.429 
[2,   600] loss: -6.200 
[2,   800] loss: -5.840 
[2,  1000] loss: -6.159 
[2,  1200] loss: -6.259 
[2,  1400] loss: -5.942 
[2,  1600] loss: -6.070 
[2,  1800] loss: -6.036 
[2,  2000] loss: -6.165 
[2,  2200] loss: -6.049 
[2,  2400] loss: -5.952 
[2,  2600] loss: -6.053 
[2,  2800] loss: -6.072 
[2,  3000] loss: -5.964 
[2,  3200] loss: -6.136 
[2,  3400] loss: -6.181 
[2,  3600] loss: -6.009 
[2,  3800] loss: -5.983 
[2,  4000] loss: -5.965 
[2,  4200] loss: -6.338 
[2,  4400] loss: -6.062 
[2,  4600] loss: -6.016 
[2,  4800] loss: -6.090 




-5.911169806644615


[3,   200] loss: -6.043 
[3,   400] loss: -5.494 
[3,   600] loss: -6.350 
[3,   800] loss: -6.028 
[3,  1000] loss: -6.153 
[3,  1200] loss: -6.428 
[3,  1400] loss: -6.109 
[3,  1600] loss: -6.129 
[3,  1800] loss: -6.151 
[3,  2000] loss: -6.329 
[3,  2200] loss: -6.226 
[3,  2400] loss: -6.220 
[3,  2600] loss: -6.153 
[3,  2800] loss: -6.278 
[3,  3000] loss: -6.232 
[3,  3200] loss: -6.376 
[3,  3400] loss: -6.362 
[3,  3600] loss: -6.367 
[3,  3800] loss: -6.139 
[3,  4000] loss: -6.288 
[3,  4200] loss: -6.404 
[3,  4400] loss: -6.292 
[3,  4600] loss: -6.024 
[3,  4800] loss: -6.373 



...


-7.218031180145689


[18,   200] loss: -7.511 
[18,   400] loss: -6.888 
[18,   600] loss: -7.719 
[18,   800] loss: -7.342 
[18,  1000] loss: -7.765 
[18,  1200] loss: -7.893 
[18,  1400] loss: -7.456 
[18,  1600] loss: -7.640 
[18,  1800] loss: -7.617 
[18,  2000] loss: -7.701 
[18,  2200] loss: -7.628 
[18,  2400] loss: -7.410 
[18,  2600] loss: -7.477 
[18,  2800] loss: -7.699 
[18,  3000] loss: -7.565 
[18,  3200] loss: -7.814 
[18,  3400] loss: -7.638 
[18,  3600] loss: -7.653 
[18,  3800] loss: -7.515 
[18,  4000] loss: -7.387 
[18,  4200] loss: -7.749 
[18,  4400] loss: -7.430 
[18,  4600] loss: -7.302 
[18,  4800] loss: -7.508 




-7.25170561007949


[19,   200] loss: -7.488 
[19,   400] loss: -7.089 
[19,   600] loss: -7.687 
[19,   800] loss: -7.434 
[19,  1000] loss: -7.733 
[19,  1200] loss: -7.914 
[19,  1400] loss: -7.591 
[19,  1600] loss: -7.655 
[19,  1800] loss: -7.729 



KeyboardInterrupt: 

In [7]:
def find_max3(tensor):
    array = tensor.cpu().detach().numpy()
    max3 = []
    for row in array:
        max3.append(np.argsort(row)[::-1][:3])
    return np.array(max3)

def compute_corrects(tensor1, tensor2):
    max_1, max_2 = find_max3(tensor1), find_max3(tensor2)
    batch_size = max_1.shape[0]
    batch_corrects = 0
    for i in range(batch_size):
        if Counter(max_1[i])==Counter(max_2[i]):
            batch_corrects+=1
    return batch_corrects

In [8]:
fs=sr
cw_len=200
cw_shift=10

wlen=int(fs*cw_len/1000.00)
wshift=int(fs*cw_shift/1000.00)




class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        cnn_arch = {
                'input_dim':wlen,
                'fs':fs,
                'cnn_N_filt':[80,60,60],
                'cnn_len_filt':[251,5,5],
                'cnn_max_pool_len':[3,3,3],
                'cnn_use_laynorm_inp':True,
                'cnn_use_batchnorm_inp':False,
                'cnn_use_laynorm':[True,True,True],
                'cnn_use_batchnorm':[False,False,False],
                'cnn_act':['leaky_relu','leaky_relu','leaky_relu'],
                'cnn_drop':[0.0,0.0,0.0]
                }
        self.cnn_net = SincNet(cnn_arch)

        dnn1_arch = {'input_dim': self.cnn_net.out_dim,
                  'fc_lay': [2048,2048,2048],
                  'fc_drop': [0.0,0.0,0.0], 
                  'fc_use_batchnorm': [True,True,True],
                  'fc_use_laynorm': [False,False,False],
                  'fc_use_laynorm_inp': False,
                  'fc_use_batchnorm_inp': False,
                  'fc_act': ['leaky_relu','leaky_relu','leaky_relu']
                  }
        self.dnn1 = MLP(dnn1_arch)


        dnn2_arch = {'input_dim':2048 ,
                  'fc_lay': [20],
                  'fc_drop': [0.0], 
                  'fc_use_batchnorm': [False],
                  'fc_use_laynorm': [False],
                  'fc_use_laynorm_inp': False,
                  'fc_use_batchnorm_inp': False,
                  'fc_act': ['linear'] # leakyrelu(1) is just identity mapping
                  }
        self.dnn2 = MLP(dnn2_arch)
        
        self.softmax = nn.Softmax(dim = 1)
        self.use_all_chunks = self.training
        
    def chop_chunk(self, signal):
        batch_size, signal_len = signal.shape
        N_fr=(signal_len-wlen)//wshift
        chunks = []
        for i in range(N_fr):
            chunks.append(signal[..., i*wshift:i*wshift+wlen]) # list of N_fr elements, each [batch_size, wlen]
        return chunks
    
    def estimate(self, chunks):
        out_vecs = []
        n_fr = len(chunks)
        batch_size, wlen = chunks[0].shape
        if not self.use_all_chunks: # ~200 chunk, each shift by 10ms, length 200 ms
            chunks = chunks[::20]
            n_fr = len(chunks)
        chunks = torch.stack(chunks, dim = 1) # [batch_size , n_fr , wlen]
        chunks = chunks.view(-1, wlen) # [batch_size*n_fr, wlen]
        out_vecs = self.softmax(self.dnn2(self.dnn1(self.cnn_net(chunks)))).clamp(min=1e-8) # [batch_size*n_fr, num_spkr]
        out_tensor = out_vecs.view(batch_size, n_fr, 20) # [batch_size, n_fr, num_spkr]
        out_tensor = out_tensor.mean(dim = 1) # batch_size*N_spkr
        return out_tensor 
    
    def forward(self, signal):
        X = self.chop_chunk(signal)
        out = self.estimate(X)
        return out

In [9]:
load_model = True

cls = Classifier().cuda(device)
optimizer = torch.optim.Adam(cls.parameters(), 0.0001)

checkpoint = torch.load('models/sincnet.pth')
cls.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#cls = nn.DataParallel(cls, device_ids = device_ids)
if 'bestacc' in checkpoint:
        bestacc = checkpoint['bestacc']

In [10]:
def cross_entropy(input, target, size_average=True):
    if size_average:
        return torch.mean(torch.sum(-target * torch.log(input), dim=1))
    else:
        return torch.sum(torch.sum(-target * torch.log(input), dim=1))

batch_size = 32
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, pin_memory = True, num_workers = 16)
#valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True, pin_memory = True, num_workers = 16)

criterion = cross_entropy

for epoch in range(64):
    tasnet.eval()
    cls.train()
    cls.use_all_chunks = False
    running_loss = 0.0
    running_accuracy = 0.0
    for batch_idx, (sig1, sig2, sig3, target1, target2, target3) in enumerate(tqdm(trainloader)):
        optimizer.zero_grad()
        sig1 = sig1.float().cuda(device)
        sig2 = sig2.float().cuda(device)
        sig3 = sig3.float().cuda(device)

        mixture = sig1+sig2+sig3
        target1 = target1.float().cuda(device)
        target2 = target2.float().cuda(device)
        target3 = target3.float().cuda(device)

        with torch.no_grad():
            estimate_source = tasnet(mixture).detach()
            loss, max_snr, estimate_source, reorder_estimate_source = \
                cal_loss(torch.stack([sig1, sig2, sig3], dim = 1), estimate_source, torch.ones(sig1.shape[0], dtype = torch.int32).cuda(device)*2*sr)
        pred1, pred2, pred3 = cls(reorder_estimate_source[:, 0]), cls(reorder_estimate_source[:, 1]), cls(reorder_estimate_source[:, 2])
        loss = cross_entropy(pred1, target1)+cross_entropy(pred2, target2)+cross_entropy(pred3, target3)
        loss.backward()
        optimizer.step()
        
        
        pred = torch.stack([pred1, pred2, pred3], dim = 0)
        pred, _ = torch.max(pred, dim = 0)
        running_loss += loss.item()
        running_accuracy += compute_corrects(pred, target1+target2+target3)/batch_size

        if batch_idx % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d]  loss: %.3f accuracy: %.3f' % 
                  (epoch + 1, batch_idx + 1, running_loss / 200, running_accuracy / 200))
            torch.save({
            'model_state_dict': cls.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'acc': running_accuracy,
            }, 'models/sincnet.pth')
            running_loss = 0.0
            running_accuracy = 0.0

[1,   200]  loss: 5.758 accuracy: 0.644
[1,   400]  loss: 5.655 accuracy: 0.692
[1,   600]  loss: 5.526 accuracy: 0.679
[1,   800]  loss: 5.457 accuracy: 0.700
[1,  1000]  loss: 5.555 accuracy: 0.660
[1,  1200]  loss: 5.608 accuracy: 0.698



[2,   200]  loss: 5.566 accuracy: 0.694
[2,   400]  loss: 5.511 accuracy: 0.724
[2,   600]  loss: 5.463 accuracy: 0.702
[2,   800]  loss: 5.292 accuracy: 0.736
[2,  1000]  loss: 5.428 accuracy: 0.693
[2,  1200]  loss: 5.522 accuracy: 0.715



[3,   200]  loss: 5.422 accuracy: 0.708
[3,   400]  loss: 5.426 accuracy: 0.740
[3,   600]  loss: 5.328 accuracy: 0.719
[3,   800]  loss: 5.326 accuracy: 0.733
[3,  1000]  loss: 5.371 accuracy: 0.727
[3,  1200]  loss: 5.441 accuracy: 0.723



[4,   200]  loss: 5.480 accuracy: 0.728
[4,   400]  loss: 5.312 accuracy: 0.745
[4,   600]  loss: 5.286 accuracy: 0.753
[4,   800]  loss: 5.333 accuracy: 0.743
[4,  1000]  loss: 5.338 accuracy: 0.733
[4,  1200]  loss: 5.336 accuracy: 0.741



[5,   200]  loss: 5.428 accuracy: 0.721
[5,   400]  loss: 5.204 accuracy: 0.759
[5,   600]  loss: 5.209 accuracy: 0.743
[5,   800]  loss: 5.236 accuracy: 0.762
[5,  1000]  loss: 5.239 accuracy: 0.739
[5,  1200]  loss: 5.245 accuracy: 0.752



...

[7,   200]  loss: 5.333 accuracy: 0.757
[7,   400]  loss: 5.185 accuracy: 0.775
[7,   600]  loss: 5.104 accuracy: 0.767
[7,   800]  loss: 5.154 accuracy: 0.778
[7,  1000]  loss: 5.152 accuracy: 0.758
[7,  1200]  loss: 5.267 accuracy: 0.768



[8,   200]  loss: 5.393 accuracy: 0.749
[8,   400]  loss: 5.169 accuracy: 0.786
[8,   600]  loss: 5.024 accuracy: 0.778
[8,   800]  loss: 5.015 accuracy: 0.784
[8,  1000]  loss: 5.032 accuracy: 0.768
[8,  1200]  loss: 5.132 accuracy: 0.775



[9,   200]  loss: 5.230 accuracy: 0.766
[9,   400]  loss: 5.053 accuracy: 0.787
[9,   600]  loss: 5.073 accuracy: 0.769
[9,   800]  loss: 5.080 accuracy: 0.794
[9,  1000]  loss: 5.097 accuracy: 0.763
[9,  1200]  loss: 5.096 accuracy: 0.780



[10,   200]  loss: 5.165 accuracy: 0.771
[10,   400]  loss: 5.038 accuracy: 0.786
[10,   600]  loss: 4.995 accuracy: 0.785
[10,   800]  loss: 5.078 accuracy: 0.794
[10,  1000]  loss: 5.045 accuracy: 0.771



KeyboardInterrupt: 

In [11]:
for i in range(5):
    batch_size = 32
    testset = OverlayDataSet('test-segments.csv')
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, pin_memory = True, num_workers = 16)
    with torch.no_grad():
        corrects = 0
        half_corrects = 0
        count = 0
        tasnet.eval()
        cls.eval()
        cls.use_all_chunks = True   
        for batch_idx, (sig1, sig2, sig3, target1, target2, target3) in enumerate(tqdm(testloader)):
            count+=sig1.shape[0]
            sig1 = sig1.float().cuda(device)
            sig2 = sig2.float().cuda(device)
            sig3 = sig3.float().cuda(device)
            mixture = sig1+sig2+sig3
            target1 = target1.float().cuda(device)
            target2 = target2.float().cuda(device)
            target3 = target3.float().cuda(device)

            estimate_source = tasnet(mixture).detach()
            loss, max_snr, estimate_source, reorder_estimate_source = \
                cal_loss(torch.stack([sig1, sig2, sig3], dim = 1), estimate_source, torch.ones(sig1.shape[0], dtype = torch.int32).cuda(device)*2*sr)
            pred1, pred2, pred3 = cls(reorder_estimate_source[:, 0]), cls(reorder_estimate_source[:, 1]), cls(reorder_estimate_source[:, 2])

            pred = torch.stack([pred1, pred2, pred3], dim = 0)
            pred, _ = torch.max(pred, dim = 0)
            corrects += compute_corrects(pred, target1+target2+target3)
        print('test accuracy: %.4f' % (corrects/len(testset)))


test accuracy: 0.7395



test accuracy: 0.7386



test accuracy: 0.7413



test accuracy: 0.7436



test accuracy: 0.7440


In [11]:
sig1, sig2, sig3, target1, target2, target3 = testset[0]
input = torch.Tensor(sig1+sig2+sig3)[None, ...].cpu()
from thop import profile
test_extractor = tasnet.module
test_extractor.cpu()
input = input.detach().cpu()
macs1, params1 = profile(test_extractor, inputs=(input, ))
print(macs1/10**9, params1/10**6)
test_discriminator = cls
test_discriminator.cpu()
macs2, params2 = profile(test_discriminator, inputs=(input,))
print(macs2/10**9, params2/10**6)
print((macs1+macs2*2)/10**9, (params1+params2)/10**6)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
[WARN] Cannot find rule for <class 'conv_tasnet.Encoder'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'conv_tasnet.ChannelwiseLayerNorm'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.activation.PReLU'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'conv_tasnet.GlobalLayerNorm'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'conv_tasnet.DepthwiseSeparableConv'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'conv_tasnet.TemporalBlock'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'conv_tasnet.TemporalConvNet'>. Treat it as zero Macs and zero Params.
[INFO] Register count_linear() for <class 'torch.nn.modules